# Importing Modules

In [1]:
from tkinter import Tk, Label, Button, Canvas, Frame, StringVar
from tkinter import TOP, BOTTOM, LEFT, RIGHT, BOTH
import time
from PIL import ImageGrab

from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np

%matplotlib inline

# Loading Trained CNN Model

In [15]:
model = load_model("digits_trained_ANN.h5")

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


# UI using Tkinter

In [19]:
class DigitsGUI:
    def __init__(self, master):
        self.master = master
        
        self.header_frame = Frame(master)
        self.header_frame.pack(side = TOP)
        
        self.canvas_frame = Frame(master)
        self.canvas_frame.pack(side = TOP)
        
        self.bottom_frame = Frame(master)
        self.bottom_frame.pack(side = TOP)
        
        # Toggles when user clicks "Enter"
        self.should_draw = False
        self.default_result = """
        Result is:
        0  : {:2.2f}%
        1  : {:2.2f}%
        2  : {:2.2f}%
        3  : {:2.2f}%
        4  : {:2.2f}%
        5  : {:2.2f}%
        6  : {:2.2f}%
        7  : {:2.2f}%
        8  : {:2.2f}%
        9  : {:2.2f}%
        """
        self.answer = StringVar()
        self.answer.set(self.default_result.format(0,0,0,0,0,0,0,0,0,0))
        
        # Coordinate lists to draw
        self.coords = []
        
        master.title("Digits classifier UI test")
        
        # Event bindings
        master.bind('<Motion>', self.motion)
        master.bind('<Key>', self.check_enter_key)
  
        # Header label
        self.label = Label(self.header_frame, text="Digits UI\nPress Enter to start\nThen draw\nPress Enter to stop")
        self.label.pack(side = TOP)
        
        # Defining Canvas
        self.C = Canvas(self.canvas_frame, bg="white", height=280, width=280)
        self.C.pack(side = LEFT)
        
        # Result UI Display
        self.summary = Label(self.canvas_frame, textvariable = self.answer)
        self.summary.pack(side = RIGHT)

        # Clear the canvas button
        self.clear_button = Button(self.bottom_frame, text="Clear", command=self.clear)
        self.clear_button.pack(side = LEFT, expand = True, fill = BOTH)
        
        # Digit Classifier Result button
        self.result_button = Button(self.bottom_frame, text="Result", command=self.result)
        self.result_button.pack(side = LEFT, expand = True, fill = BOTH)
        
        # Close button
        self.close_button = Button(self.bottom_frame, text="Close", command=self.close)
        self.close_button.pack(side = LEFT, expand = True, fill = BOTH)

    # Toggles when user press enter key in keyboard
    def check_enter_key(self, event):
        if event.keycode == 13: # 13 is key code for "Enter"
            self.should_draw ^= True
            print("Toggled to: {}".format(self.should_draw))
            self.coords = []
    
    # Cleaning the canvas function
    def clear(self):
        print("Clearing!")
        self.C.delete("all")
        self.coords = []
        self.answer.set(self.default_result.format(0,0,0,0,0,0,0,0,0,0))
        
    # Classifier result function
    def result(self):
        print("Saving Image")
        self.getter()
        
        print("Loading image")
        img = cv2.imread("canvas_image.jpg")
        
        print("Preprocessing")
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        res = cv2.resize(gray, (28,28))
        cv2.imwrite("res.jpg", res)
        res = res.astype(np.float64)
        res = res/255.0
        res = res.reshape((784,))
        print(res.shape, res.dtype, res.max(), res.min())
        
        print("Prediction")
        print(np.array([res]).shape, np.array([res]).dtype, np.array([res]).max())
#         with open("output.txt", "wb") as file:
#             file.write(np.array([res]).tostring())
        prediction = model.predict(np.array([res]))
        print(prediction)
        self.answer.set(self.default_result.format(*prediction[0]))
        
    # Closing the GUI function
    def close(self):
        print("Closing!")
        self.master.destroy()
    
    # Saving the canvas as image on "Result" button click
    def getter(self):
        x=self.canvas_frame.winfo_rootx() + self.C.winfo_x()
        y=self.canvas_frame.winfo_rooty() + self.C.winfo_y()
        x1=x+self.C.winfo_width()
        y1=y+self.C.winfo_height()
        
        ImageGrab.grab().crop((x,y,x1,y1)).save("canvas_image.jpg")
        
    # Draws lines on canvas
    def motion(self, event):
        if self.should_draw == False:
            return
        time.sleep(0.25)
        x, y = event.x, event.y
        if self.coords == []:
            self.coords.extend([x,y, x,y])
        else:
            self.coords.extend([x,y])
        line = self.C.create_line(self.coords,fill='black', width=15)
        y = self.coords.pop()
        x = self.coords.pop()
        self.coords = [x,y]
        return
        

root = Tk()
my_gui = DigitsGUI(root)
root.mainloop()

Toggled to: True
Toggled to: False
Saving Image
Loading image
Preprocessing
(784,) float64 1.0 0.0
Prediction
(1, 784) float64 1.0
[[5.4484878e-10 9.2727900e-17 5.6392772e-05 2.9228633e-02 7.1847018e-16
  8.5006322e-06 7.9851151e-01 1.7219488e-01 1.0737023e-17 1.3224130e-21]]
Clearing!
Toggled to: True
Toggled to: False
Saving Image
Loading image
Preprocessing
(784,) float64 1.0 0.0
Prediction
(1, 784) float64 1.0
[[1.4438511e-13 9.8786620e-17 2.5208008e-05 6.1650909e-03 1.5460190e-16
  3.6651067e-09 3.3959422e-02 9.5985031e-01 3.9218789e-19 4.3408337e-23]]
Clearing!
Toggled to: True
Toggled to: False
Saving Image
Loading image
Preprocessing
(784,) float64 1.0 0.0
Prediction
(1, 784) float64 1.0
[[1.19413657e-09 2.68089931e-17 2.58633809e-05 1.26543455e-02
  2.12055071e-14 9.10151357e-07 7.96734929e-01 1.90583900e-01
  8.38765939e-18 2.87029330e-21]]
Clearing!
Toggled to: True
Saving Image
Loading image
Preprocessing
(784,) float64 1.0 0.0
Prediction
(1, 784) float64 1.0
[[8.8764472e-1